## The Metrics

Before we move onto the model, and how is trained and tested, let's quickly go through the metrics that we will use here. The first part of the code below is either a direct copy/paste from the original [repo](https://github.com/xiangwang1223/neural_graph_collaborative_filtering) or a minor adaptation. When this is not the case I will explain the corresponding details. Therefore, **all credit to the authors** (Xiang Wang, Xiangnan He, Meng Wang, Fuli Feng and Tat-Seng Chua).

In [1]:
import numpy as np
import heapq

from sklearn.metrics import roc_auc_score

In [2]:
r = np.random.choice(2, 20, p=[0.8, 0.2])
k = 10
n_inter = 20

In [3]:
r

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [4]:
def recall_at_k(r, k, n_inter):
    """recall @ k
    Parameters:
    ----------
    r: Iterable
        binary iterable (nonzero is relevant).
    k: Int
        number of recommendations to consider
    n_inter: Int
        number of interactions
    Returns:
    ----------
    recall @ k
    """
    r = np.asfarray(r)[:k]
    return np.sum(r) / n_inter

In [5]:
recall_at_k(r, k, n_inter)

0.1

In [6]:
def precision_at_k(r, k):
    """precision @ k
    Parameters:
    ----------
    r: Iterable
        binary iterable (nonzero is relevant).
    k: Int
        number of recommendations to consider
    Returns:
    ----------
    Precision @ k
    """
    assert k >= 1
    r = np.asarray(r)[:k]
    return np.mean(r)

In [7]:
precision_at_k(r, k)

0.2

In [8]:
def dcg_at_k(r, k, method=1):
    """ discounted cumulative gain (dcg) @ k
    Parameters:
    ----------
    r: Iterable
        Relevance is positive real values. If binary, nonzero is relevant.
    k: Int
        number of recommendations to consider
    method: Int
        one of 0 or 1. Simply, different dcg implementations
    Returns:
    ----------
    dcg @ k
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.

In [9]:
dcg_at_k(r, k)

1.1309297535714575

In [10]:
dcg_at_k(r, k, method=0)

1.6309297535714575

In [11]:
def ndcg_at_k(r, k, method=1):
    """ Normalized discounted cumulative gain @ k
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [12]:
ndcg_at_k(r, k)

0.5307212739772434

In [13]:
def hit_at_k(r, k):
    """hit ratio @ k
    Parameters:
    ----------
    r: Iterable
        binary iterable (nonzero is relevant).
    k: Int
        number of recommendations to consider
    Returns:
    ----------
    hit ratio @ k
    """
    r = np.array(r)[:k]
    if np.sum(r) > 0:
        return 1.
    else:
        return 0.

In [14]:
hit_at_k(r,k)

1.0

In [15]:
def get_auc(item_score, user_pos_test):
    """Wrap up around sklearn's roc_auc_score
    Parameters:
    ----------
    item_score: Dict
        Dict. keys are item_ids, values are predictions
    user_pos_test: List
        List with the items that the user actually interacted with
    Returns:
    ----------
    res: Float
        roc_auc_score
    """
    item_score = sorted(item_score.items(), key=lambda kv: kv[1])
    item_score.reverse()
    item_id = [x[0] for x in item_score]
    score = [x[1] for x in item_score]

    r = []
    for i in item_id:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)

    try:
        res = roc_auc_score(r, score)
    except Exception:
        res = 0.

    return res

Let's build the inputs of the function:

In [16]:
# for example...let's assume 100 items in total
item_score = {k:v for k,v in zip(np.arange(100), np.random.rand(100))}
user_pos_test = np.random.choice(100, 20, replace=False)

In [17]:
item_id = 1
item_score[item_id]

0.3779146754344742

In [18]:
print(user_pos_test)

[47 72 93 54 32 71 57 20 92 96 29 25 70 63 99  5 82 30 43  0]


In [19]:
get_auc(item_score, user_pos_test)

0.475

In [20]:
def auc(true, pred):
    """Simple wrap up around sklearn's roc_auc_score
    """
    try:
        res = roc_auc_score(true, pred)
    except Exception:
        res = 0.
    return res


def ranklist_by_sorted(user_pos_test, test_items, rating, Ks):
    """
    Retursn a binary list, where relevance is nonzero, based on a ranked list
    with the n largest scores. Also returns the AUC
    Parameters:
    ----------
    user_pos_test: List
        List with the items that the user actually interacted with
    test_items: List
        List with the all items in the test dataset
    rating: List
        List with the ratings corresponding to test_items
    Ks: Int or List
        the k in @k
    Returns:
    ----------
    r: binary list where nonzero in relevant
    auc: testing roc_auc_score
    """
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = get_auc(item_score, user_pos_test)
    return r, auc

In [21]:
test_items, rating = list(item_score.keys()), list(item_score.values())
Ks = [5,10]

In [22]:
ranklist_by_sorted(user_pos_test, test_items, rating, Ks)

([0, 0, 0, 0, 0, 1, 1, 0, 0, 0], 0.475)

In [23]:
def ranklist_by_heapq(user_pos_test, test_items, rating, Ks):
    """
    Retursn a binary list, where relevance is nonzero, based on a ranked list
    with the n largest scores. For consistency with ranklist_by_sorted, also
    returns auc=0 (since auc does not make sense within a mini batch)
    Parameters:
    ----------
    user_pos_test: List
        List with the items that the user actually interacted with
    test_items: List
        List with the all items in the test dataset
    rating: List
        List with the ratings corresponding to test_items
    Ks: Int or List
        the k in @k
    Returns:
    ----------
    r: binary list where nonzero in relevant
    """
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = 0.
    return r, auc

In [24]:
ranklist_by_heapq(user_pos_test, test_items, rating, Ks)

([0, 0, 0, 0, 0, 1, 1, 0, 0, 0], 0.0)

And finally, getting altogether:

In [25]:
def get_performance(user_pos_test, r, auc, Ks):
    """wrap up around all other previous functions
    ----------
    user_pos_test: List
        List with the items that the user actually interacted with
    r: List
        binary list where nonzero in relevant
    auc: Float
        sklearn's roc_auc_score
    Ks: List
        the k in @k
    Returns:
    ----------
    dictionary of metrics
    """

    precision, recall, ndcg, hit_ratio = [], [], [], []

    for K in Ks:
        precision.append(precision_at_k(r, K))
        recall.append(recall_at_k(r, K, len(user_pos_test)))
        ndcg.append(ndcg_at_k(r, K))
        hit_ratio.append(hit_at_k(r, K))

    return {'recall': np.array(recall), 'precision': np.array(precision),
            'ndcg': np.array(ndcg), 'hit_ratio': np.array(hit_ratio), 'auc': auc}

In [26]:
r, auc = ranklist_by_sorted(user_pos_test, test_items, rating, Ks)

In [27]:
get_performance(user_pos_test, r, auc, Ks)

{'recall': array([0. , 0.1]),
 'precision': array([0. , 0.2]),
 'ndcg': array([0.        , 0.42278983]),
 'hit_ratio': array([0., 1.]),
 'auc': 0.475}

### GPU test

Before we leave the metrics behind, let's pause for one second and have a look to the functions above. They all provide scores/metrics for one user. This means that this will have run in a loop or distributed over the cores of the machine where we run the algorithm. Given the fact that the algorithm will run on a GPU, maybe we could take advantage and write some evaluation function that runs on the GPU. 

The code below is taken mostly from [here](https://github.com/sh0416/bpr/blob/master/train.py), adapated to the fact that here our rating matrix is large enough so that it wont fit in memory when move to dense (i.e. we cannot run lines like: `test_pred_mask = 1 - (train_w)` in that code).

In [28]:
import torch
import scipy.sparse as sp

In [29]:
use_cuda = torch.cuda.is_available()

n_users = 100
n_items = 200
n_embed = 12
Ks=[5,10]

let's create some small, fake dataset to illustrate the use of this testing method

In [30]:
# user and item embeddings
user_emb = torch.from_numpy(np.random.rand(n_users, n_embed))
item_emb = torch.from_numpy(np.random.rand(n_items, n_embed))

In [31]:
# Train Ratings Matrix
def randbin(r,c,p):
    return np.random.choice([0, 1], size=(r,c), p=[p, 1-p])
R_tr = randbin(n_users, n_items, 0.8)

In [32]:
# Test Rating Matrix
# removing all items in training
temp_mtx = 1 - R_tr
# finding the corresponding indexes
temp_idx = np.where(temp_mtx)
# setting the testing size as, for example, training//5
test_fr = np.where(R_tr)[0].size//5
# chosing indexes at random
R_te_idx = np.random.choice(temp_idx[0].size, test_fr, replace=False)
i,j = temp_idx[0][R_te_idx], temp_idx[1][R_te_idx]
# setting them to 1
R_te = np.zeros((n_users, n_items))
R_te[i,j] = 1

When we run the "real thing" `R_tr` and `R_te` will be sparse matrices

In [33]:
R_tr = sp.csr_matrix(R_tr, dtype='float64')
R_te = sp.csr_matrix(R_te, dtype='float64')

In [34]:
def split_mtx(X, n_folds=10):
    """
    Split a matrix/Tensor into n_folds    
    """
    X_folds = []
    fold_len = X.shape[0]//n_folds
    for i in range(n_folds):
        start = i * fold_len
        if i == n_folds -1:
            end = X.shape[0]
        else:
            end = (i + 1) * fold_len
        X_folds.append(X[start:end])
    return X_folds

And this is the testing function

In [35]:
def precision_and_recall_k(user_emb, item_emb, R_tr, R_te, Ks):
    """
    Compute precision and recall using tensors    

    Parameters:
    ----------
    user_emb: Tensor
        user embeddings of shape (n_users, n_emb)
    item_emb: Tensor
        item embeddings of shape (n_items, n_emb)
    R_tr: scipy.sp matrix
        training ratings shape (n_users, n_items)
    R_te: scipy.sp matrix
        testing ratings shape (n_users, n_items)
    Ks: List
        k order of recommendations (the k in precision@k)

    Returns:
    ----------
    precision[k],recall[k]: Dict
        Dictionary where keys are the Ks and values are precision and recall
    """
    # splits into n_folds
    tr_folds = split_mtx(R_tr)
    te_folds = split_mtx(R_te)
    ue_folds = split_mtx(user_emb)    
    
    fold_prec, fold_rec = {}, {}
    for ue_fold, tr_fold, te_fold in zip(ue_folds, tr_folds, te_folds):
        
        # score for all items, per user. (the author of the code used sigmoid so we'll leave it there)
        result = torch.sigmoid(torch.mm(ue_fold, item_emb.t()))
        # this mask contains all that is not training (negatives+testing)
        test_pred_mask = torch.from_numpy(1 - tr_fold.todense())
        # this mask contains only the true testing items
        test_true_mask = torch.from_numpy(te_fold.todense())
        test_pred_mask, test_true_mask = test_pred_mask.cuda(), test_true_mask.cuda()
        test_pred = test_pred_mask * result.cuda()
        test_true = test_true_mask * result.cuda()

        _, test_indices = torch.topk(test_pred, dim=1, k=max(Ks))
        for k in Ks:
            topk_mask = torch.zeros_like(test_pred)
            source = torch.tensor(1.0).cuda() if use_cuda else torch.tensor(1.0)
            # this will create a mask with 1 located in locations test_indices[:, :k]
            topk_mask.scatter_(dim=1, index=test_indices[:, :k], src=source)
            # matrix with the actual predictions in locations test_indices[:, :k]
            test_pred_topk = topk_mask * test_pred
            # precision@k and recall@k per fold
            acc_result = (test_pred_topk != 0) & (test_pred_topk == test_true)
            pr_k = acc_result.sum().float() / (user_emb.shape[0] * k)
            rec_k = (acc_result.float().sum(dim=1) / test_true_mask.float().sum(dim=1))
            try:
                fold_prec[k].append(pr_k)
                fold_rec[k].append(rec_k)
            except KeyError:
                fold_prec[k] = [pr_k]
                fold_rec[k] = [rec_k]

    precision, recall = {}, {}
    for k in Ks:
        precision[k] = np.sum(fold_prec[k])
        recall[k] = torch.cat(fold_rec[k]).mean()
    return precision,recall

Let's have a look to what happens inside that function

In [36]:
tr_folds = split_mtx(R_tr)
te_folds = split_mtx(R_te)
ue_folds = split_mtx(user_emb)

In [37]:
tr_folds[0]

<10x200 sparse matrix of type '<class 'numpy.float64'>'
	with 384 stored elements in Compressed Sparse Row format>

In [38]:
print(len(ue_folds), ue_folds[0].shape)

10 torch.Size([10, 12])


now we have 10 folds/partition of the rating and user embedding matrices and we are ready to loop. Let's got through one loop

In [39]:
tr_fold, te_fold, ue_fold = tr_folds[0], te_folds[0], ue_folds[0]

the authors explain that they want to make all scores between 0 and 1, using a sigmoid. Below are the score for all items, for the N users in the corresponding fold

In [40]:
result = torch.sigmoid(torch.mm(ue_fold, item_emb.t()))
print(result.shape)

torch.Size([10, 200])


In [41]:
# masks with 1 for all items that are NOT in training -> test+negatives
test_pred_mask = torch.from_numpy(1 - tr_fold.todense())
# masks with 1 for test items (is a copy of R_te per fold)
test_true_mask = torch.from_numpy(te_fold.todense())

In [42]:
# matrix with scores for all items that are NOT in training -> test+negatives
test_pred = test_pred_mask * result
# matrix with scores for "true" test items 
test_true = test_true_mask * result

In [43]:
print(test_pred.shape, test_true.shape)

torch.Size([10, 200]) torch.Size([10, 200])


In [44]:
test_pred

tensor([[0.0000, 0.9600, 0.0000,  ..., 0.9339, 0.9849, 0.9713],
        [0.9741, 0.0000, 0.9946,  ..., 0.9459, 0.9943, 0.0000],
        [0.8255, 0.8635, 0.9064,  ..., 0.7081, 0.8886, 0.0000],
        ...,
        [0.9508, 0.9203, 0.9889,  ..., 0.8648, 0.0000, 0.9543],
        [0.9379, 0.9558, 0.9828,  ..., 0.9271, 0.9786, 0.9824],
        [0.9608, 0.8637, 0.9727,  ..., 0.8701, 0.9637, 0.8462]],
       dtype=torch.float64)

In [45]:
test_true

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

Let's find the locations of the top K recommended items

In [46]:
_, test_indices = torch.topk(test_pred, dim=1, k=max(Ks))

In [47]:
test_indices

tensor([[ 95,   9,  13,  92, 173, 151,  41, 168,  93,   6],
        [173, 108,   9,  95,  13, 151,   2,  91,  92, 198],
        [  9, 148,  92,  95, 174,  60,  68, 166,  13, 112],
        [ 92,   9, 173, 134, 108,  93, 195,  24,   7,  78],
        [134, 173,   6, 152, 108,  92,  18, 142,  37,  78],
        [ 95, 151,   6, 173, 168,  92,  93,   2, 148, 138],
        [  6, 173, 134, 148, 100,  13, 161, 174,  41,  92],
        [ 95,   2,   9, 173, 134, 151,  92, 139, 183,  13],
        [173,   9,  95,  13,  93,  17,  77, 134, 147, 109],
        [  9,   2, 134, 148, 168,  13,   6, 100, 152,  91]])

let's assume k=5

In [48]:
k=5
topk_mask = torch.zeros_like(test_pred)
source = torch.tensor(1.0).cuda() if use_cuda else torch.tensor(1.0)
topk_mask.scatter_(dim=1, index=test_indices[:, :k], src=source)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [49]:
topk_mask[0,].nonzero()

tensor([[  9],
        [ 13],
        [ 92],
        [ 95],
        [173]])

effectively, the nonzero locations in the first row correspond to the top 5 items in test_indices. Let's get these locations from the `test_pred` tensor and compute the precision and recall (or almost)  

In [50]:
test_pred_topk = topk_mask * test_pred 

In [51]:
# if item in not in training and is in testing (i.e. not negative)
acc_result = (test_pred_topk != 0) & (test_pred_topk == test_true)

We save the neccesary information per fold that will be used to calculate precision and recall for the whole dataset

In [52]:
pr_k = acc_result.sum().float() / (user_emb.shape[0] * k)
rec_k = (acc_result.float().sum(dim=1) / test_true_mask.float().sum(dim=1))

In [53]:
pr_k

tensor(0.0100)

In [54]:
rec_k

tensor([0.0000, 0.0000, 0.0000, 0.2000, 0.0000, 0.0833, 0.1250, 0.1111, 0.0000,
        0.0000])

In [55]:
fold_prec, fold_rec = {}, {}
try:
    fold_prec[k].append(pr_k)
    fold_rec[k].append(rec_k)
except KeyError:
    fold_prec[k] = [pr_k]
    fold_rec[k] = [rec_k]

In [56]:
print(fold_prec)
print(fold_rec)

{5: [tensor(0.0100)]}
{5: [tensor([0.0000, 0.0000, 0.0000, 0.2000, 0.0000, 0.0833, 0.1250, 0.1111, 0.0000,
        0.0000])]}


Finally (remember, this would run into a loop of Ks)

In [57]:
precision, recall = {}, {}
precision[k] = np.sum(fold_prec[k])
recall[k] = torch.cat(fold_rec[k]).mean()

In [58]:
precision

{5: 0.01}

In [59]:
recall

{5: tensor(0.0519)}

Let's run the whole thing

In [60]:
precision, recall = precision_and_recall_k(user_emb, item_emb, R_tr, R_te, Ks=[5, 10])

In [61]:
print(precision, recall)

{5: tensor(0.0480, device='cuda:0'), 10: tensor(0.0440, device='cuda:0')} {5: tensor(0.0269, device='cuda:0'), 10: tensor(0.0491, device='cuda:0')}


Note that in the final version of the code, I will rename `precision_and_recall_k` to `test_GPU` as "opposed" to the [Wang Xiang et al](https://arxiv.org/pdf/1905.08108.pdf) paper test funcion, which I will refer as `test_CPU`